In [ ]:
#####################################################imports

In [ ]:
!pip install pandas

In [ ]:
import os
import pandas as pd

X = pd.read_csv('train.csv', sep=',', encoding='ISO-8859-1')
y = X['evaluator_rating']
X = X.dropna(axis=1)
X = X.drop(columns=['Unnamed: 0', 'uniqueId','evaluator_rating'])


In [ ]:
X.head()

,promptId,essay
0,1,"At present age, our education system is not go..."
1,1,I am agree the tightly defined curriculum of o...
2,1,I strongly agree with the statement that tight...
3,1,Our education system is nice quitely but i dis...
4,1,i am totally agree with the statement that tig...


In [ ]:
X

,promptId,essay
0,1,"At present age, our education system is not go..."
1,1,I am agree the tightly defined curriculum of o...
2,1,I strongly agree with the statement that tight...
3,1,Our education system is nice quitely but i dis...
4,1,i am totally agree with the statement that tig...
...,...,...
1235,5,The entire world is in the race of producing a...
1236,5,The race in the development of weapons are pro...
1237,5,In an era where every second person hopes and ...
1238,5,INTRODUCTION :Since the beginning of the time ...


In [ ]:
######################################### Preprocessors and word2vec functions

In [ ]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec

def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [ ]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
################### LSTM MODEL

In [ ]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
minimum_scores = [0]*1240
maximum_scores = [5]*1240

In [ ]:
########################## TRAIN

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score

cv = KFold(n_splits = 5, shuffle = True)
results = []
y_pred_list = []
y_test_list = []
arr = [0]*248


count = 1

for traincv, testcv in cv.split(X):
    print("\n--------Fold {}--------\n".format(count))
    X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
    
    train_essays = X_train['essay']
    test_essays = X_test['essay']

    sentences = []

    y_test_list = [0]*248

    for i in range(0,248):
        y_test_list[i] = int(np.around(y_test.values[i]))

    for essay in train_essays:
         # Obtaining all sentences from the training essays.
         sentences += essay_to_sentences(essay, remove_stopwords = True)
            
    # Initializing variables for word2vec model.
    num_features = 300 
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3

    print("Training Word2Vec Model...")
    model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

    model.init_sims(replace=True)
    model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)
    clean_train_essays = []

    # Generate training and testing data word vectors.
    for essay_v in train_essays:
        clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))
    
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)

    clean_test_essays = []
    for essay_v in test_essays:
        clean_test_essays.append(essay_to_wordlist( essay_v, remove_stopwords=True ))
    testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )

    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))

    lstm_model = get_model()
    lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=2)
    #lstm_model.load_weights('./model_weights/final_lstm.h5')
    y_pred = lstm_model.predict(testDataVecs)
    
    print(y_pred)
    
    # Save models.
    lstm_model.save('final_lstm%s.h5' %count)
    
    for i in range(0,248):
        arr[i] = int(np.around(y_pred[i][0]))
    

    # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
    #result = cohen_kappa_score(y_test_list,arr,weights='quadratic')
    print("Kappa Score: {}".format(result))
    #results.append(result)
    y_pred_list = []
    count += 1


--------Fold 1--------

Training Word2Vec Model...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_38 (LSTM)               (None, 1, 300)            721200    
_________________________________________________________________
lstm_39 (LSTM)               (None, 64)                93440     
_________________________________________________________________
dropout_19 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 65        
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
16/16 [==============================] - 7s 23ms/step - loss: 5.7361 - mae: 2.1181
Epoch 2/2
16/16 [==============================] - 0s 22ms/step - loss: 1.2512 - mae: 0.8574
[[2.7756655]
 [2.650485 ]
 [2.7978015]
 [2.9944186]
 [2.732

In [ ]:
################################ inference

In [ ]:
import os
import pandas as pd

X = pd.read_csv('test.csv', sep=',', encoding='ISO-8859-1')
#y = X['evaluator_rating']
X = X.dropna(axis=1)
X = X.drop(columns=['Unnamed: 0', 'uniqueId'])

In [ ]:
X.essay

0      Curriculum has been adopted in many schools. T...
1      I strongly agree with the statement ,  The tig...
2      Imagination and creativity is the most importa...
3      In our eduction system leaves no room for imag...
4      I will agree at some what extend, because if w...
                             ...                        
300    Earth is a creation of God and everything that...
301    production of arms and weapons in this present...
302    Race to become more powerful can destroy the e...
303    In its attempt to harness the power of the ato...
304    Racein the production of arms and weapons in t...
Name: essay, Length: 305, dtype: object

In [ ]:
cv = KFold(n_splits = 5, shuffle = True)


#X_train = X.iloc[traincv]
    
train_essays = X['essay']

In [ ]:
train_essays

0      Curriculum has been adopted in many schools. T...
1      I strongly agree with the statement ,  The tig...
2      Imagination and creativity is the most importa...
3      In our eduction system leaves no room for imag...
4      I will agree at some what extend, because if w...
                             ...                        
300    Earth is a creation of God and everything that...
301    production of arms and weapons in this present...
302    Race to become more powerful can destroy the e...
303    In its attempt to harness the power of the ato...
304    Racein the production of arms and weapons in t...
Name: essay, Length: 305, dtype: object

In [ ]:
X['essay'][0]

'Curriculum has been adopted in many schools. This curriculum had a great impact on the developement of the children.But nowadays we see that the education institutions had tightened the curriculum system in such a manner that the children andyoung people arenot getting enough time to be indulged in other activities. Due to the strict and tightened rules the capacity of imagination and creativity isaffected badly.They become Book worm.A good curriculum is good for the children but apart from this same preference should be given to other activities.Following curriculum is necessary but education system have to understand the need of creativity and imagination role in a life of a person.There should a enough amount of curriculum should be prepared by the education systems that it will not affect the children and he or she will not feel the cumbersome of the theories.Due to the tightened curriculum thereb is no space left for the imagination and creativity. A creative mind is better than 

In [ ]:

from keras.models import load_model

train_essays = X['essay']

sentences = []

for essay in train_essays:
    # Obtaining all sentences from the training essays.
    sentences += essay_to_sentences(essay, remove_stopwords = True)

print(sentences)

# Initializing variables for word2vec model.
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

#model.init_sims(replace=True)

clean_train_essays = []

# Generate training and testing data word vectors.
for essay_v in train_essays:
    clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))

print(clean_train_essays)

trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)

trainDataVecs = np.array(trainDataVecs)

trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))

print(trainDataVecs)

lstm_model = load_model("final_lstm5.h5")

y_pred = lstm_model.predict(trainDataVecs)

    

In [ ]:
y_pred

305

In [ ]:
liist = []
for i in range(0,304):
    liist.append(np.round(y_pred[i][0]-2,2))

In [ ]:
import csv

In [ ]:
X = pd.read_csv('test.csv', sep=',', encoding='ISO-8859-1')

In [ ]:
file = open("test_prediction.csv", "w")
writer = csv.writer(file)

for w in range(0,304):

    writer.writerow([X['promptId'][w],X['uniqueId'][w],X['essay'][w], liist[w]])

file.close()


In [ ]:
liist